In [1]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn import linear_model
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
import math

# This LAB will be improved this week, not satisfied
# Still some study required

In [2]:
#fifa = pd.read_csv(r'/Users/larag/Documents/Ironhack/Fifa_project/fifa21_train.csv')
# fifa = pd.DataFrame(pd.read_csv('fifa21_train.csv'))
fifa = pd.DataFrame(pd.read_csv('fifa21_validate.csv'))
print(fifa.shape)    # before validate: (11701, 101)

(1999, 101)


In [3]:
fifa = fifa.drop(columns=["ID","Club"
                          ,"Position"
                          ,"Team & Contract"
                          ,"Joined"
                          ,"Loan Date End"
                          ,"Release Clause"
                          ,"Contract"
                          ,"Growth"
                          ,"A/W"
                          ,"D/W"
                          ,'LS'
                          ,'ST',
                         'RS',
                         'LW',
                         'LF',
                         'CF',
                         'RF',
                         'RW',
                         'LAM',
                         'CAM',
                         'RAM',
                         'LM',
                         'LCM',
                         'CM',
                         'RCM',
                         'RM',
                         'LWB',
                         'LDM',
                         'CDM',
                         'RDM',
                         'RWB',
                         'LB',
                         'LCB',
                         'CB',
                         'RCB',
                         'RB',
                         'GK'
                         ,"W/F",
                          "SM",
                          "IR",
                          "Value",
                          "Name",
                          "Nationality",
                          "foot",
                          "Hits"])

In [4]:
fifa['Wage'] = (pd.to_numeric(fifa['Wage'].str.strip('K').str.strip('€')))* 1000

In [5]:
fifa['Weight']=pd.to_numeric(fifa['Weight'].str.strip('lbs'))

In [6]:
fifa['Height']=((pd.to_numeric(fifa['Height'].str[0]))*12)+ (pd.to_numeric(fifa['Height'].str.split("\'").str[1].str.strip('"')))*2.54

In [7]:
fifa = fifa.dropna()

In [8]:
# GK = 0, Defense = 1, Midfield = 2, Offense = 3

fifa["BP"]=(fifa["BP"]
            .replace("GK",0)
            .replace("LB",1)
            .replace("CB",1)
            .replace("RB",1)
            .replace("ST",3)
            .replace("RW",3)
            .replace("LWB",3)
            .replace("LW",3)
            .replace("RWB",3)
            .replace("CDM",2)
            .replace("CAM",2)
            .replace("LM",2)
            .replace("RM",2)
            .replace("CM",2)
            .replace("CF",3))

In [9]:
fifa.dtypes

Age                   int64
BP                    int64
Height              float64
Weight                int64
Wage                  int64
Attacking             int64
Crossing              int64
Finishing             int64
Heading Accuracy      int64
Short Passing         int64
Volleys             float64
Skill                 int64
Dribbling             int64
Curve               float64
FK Accuracy           int64
Long Passing          int64
Ball Control          int64
Movement              int64
Acceleration          int64
Sprint Speed          int64
Agility             float64
Reactions             int64
Balance             float64
Power                 int64
Shot Power            int64
Jumping             float64
Stamina               int64
Strength              int64
Long Shots            int64
Mentality             int64
Aggression            int64
Interceptions       float64
Positioning         float64
Vision              float64
Penalties             int64
Composure           

In [10]:
fifa.corr()

,Age,BP,Height,Weight,Wage,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,...,GK Reflexes,Total Stats,Base Stats,PAC,SHO,PAS,DRI,DEF,PHY,OVA
Age,1.000000,-0.093309,-0.049150,0.245415,-0.250552,0.161731,0.124160,0.082438,0.154803,0.152380,...,0.132964,0.265483,0.406148,-0.175650,0.293637,0.384012,0.224738,0.225096,0.440184,0.511036
BP,-0.093309,1.000000,0.085510,-0.327754,-0.021139,0.715425,0.507804,0.807375,0.427856,0.515040,...,-0.622882,0.502421,0.033797,0.202646,0.313094,0.013020,0.203867,-0.329542,-0.188292,-0.016096
Height,-0.049150,0.085510,1.000000,-0.137608,-0.005397,0.102739,0.166938,0.108974,-0.042691,0.096120,...,-0.080130,0.106511,0.070427,0.145626,0.054551,0.088932,0.104460,-0.029081,-0.097900,-0.006823
Weight,0.245415,-0.327754,-0.137608,1.000000,-0.041336,-0.301580,-0.416321,-0.326636,0.045686,-0.293241,...,0.353489,-0.271523,0.011090,-0.339818,-0.015385,-0.088039,-0.201508,0.128639,0.522357,0.153761
Wage,-0.250552,-0.021139,-0.005397,-0.041336,1.000000,-0.179943,-0.153441,-0.122357,-0.161163,-0.193318,...,0.028547,-0.254430,-0.296389,-0.048704,-0.148409,-0.223261,-0.209494,-0.165037,-0.229778,-0.344150
Attacking,0.161731,0.715425,0.102739,-0.301580,-0.179943,1.000000,0.853797,0.881579,0.706756,0.887668,...,-0.772465,0.906113,0.391315,0.165308,0.313100,0.338772,0.333692,0.127482,0.074592,0.388059
Crossing,0.124160,0.507804,0.166938,-0.416321,-0.153441,0.853797,1.000000,0.673482,0.452006,0.803063,...,-0.664722,0.854196,0.444607,0.294450,0.188991,0.465588,0.402335,0.216285,-0.025786,0.347164
Finishing,0.082438,0.807375,0.108974,-0.326636,-0.122357,0.881579,0.673482,1.000000,0.445152,0.666363,...,-0.585434,0.716262,0.335145,0.269165,0.580332,0.318889,0.443499,-0.242397,-0.110508,0.263854
Heading Accuracy,0.154803,0.427856,-0.042691,0.045686,-0.161163,0.706756,0.452006,0.445152,1.000000,0.643519,...,-0.731046,0.619031,0.075322,-0.179220,-0.168480,-0.133289,-0.195308,0.408459,0.386677,0.269760
Short Passing,0.152380,0.515040,0.096120,-0.293241,-0.193318,0.887668,0.803063,0.666363,0.643519,1.000000,...,-0.734279,0.907076,0.437951,0.083052,0.123721,0.426411,0.303432,0.377462,0.139720,0.456423


In [11]:
fifa = fifa.drop(columns=['Height','Weight','Acceleration','Sprint Speed','Agility','Balance','Jumping',
                 'Interceptions','Defending','Marking','Standing Tackle', 'Sliding Tackle', 'Goalkeeping',
                 'GK Diving','GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes', 'PAC', 'DRI','DEF','PHY'])               

In [12]:
y = fifa["OVA"]
X = fifa.drop(columns=["OVA"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
lm = linear_model.LinearRegression() 
lm.fit(X_train,y_train)

LinearRegression()

In [14]:
from sklearn.preprocessing import MinMaxScaler

transformer = MinMaxScaler().fit(X)
x_normalized = transformer.transform(X)
print(x_normalized.shape)
df1 = pd.DataFrame(x_normalized, columns=X.columns)   # (11422, 32)

(1949, 32)


In [15]:
predictions = lm.predict(X_train)

r2 = r2_score(y_train, predictions)
mse = mean_squared_error(y_train, predictions)
rmse = math.sqrt(mse)
mae = mean_absolute_error(y_train, predictions)

print('r2_train:', r2)
print('mse_train:', mse)
print('rmse_train:', rmse)
print('mae_train:', mae)

# Before
# r2_train: 0.8982606709876451
# mse_train: 4.76644433400382
# rmse_train: 2.183218801220762
# mae_train: 1.7172527676690437

r2_train: 0.9010451300643276
mse_train: 4.551572165705338
rmse_train: 2.133441390267222
mae_train: 1.6957644298090853


In [16]:
predictions = lm.predict(X_test)

r2 = r2_score(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = math.sqrt(mse)
mae = mean_absolute_error(y_test, predictions)

print('r2_test:', r2)
print('mse_test:', mse)
print('rmse_test:', rmse)
print('mae_test:', mae)

# Before:
# r2_test: 0.8892247854669821
# mse_test: 5.189059654187919
# rmse_test: 2.277950757630182
# mae_test: 1.7609707766112719

r2_test: 0.8843279716841483
mse_test: 4.873134028470924
rmse_test: 2.2075176167974115
mae_test: 1.768202267989874


In [17]:
def fifa_processing(fifa):
    fifa = fifa.drop(columns=["ID","Club"
                          ,"Position"
                          ,"Team & Contract"
                          ,"Joined"
                          ,"Loan Date End"
                          ,"Release Clause"
                          ,"Contract"
                          ,"Growth"
                          ,"A/W"
                          ,"D/W"
                          ,'LS'
                          ,'ST',
                         'RS',
                         'LW',
                         'LF',
                         'CF',
                         'RF',
                         'RW',
                         'LAM',
                         'CAM',
                         'RAM',
                         'LM',
                         'LCM',
                         'CM',
                         'RCM',
                         'RM',
                         'LWB',
                         'LDM',
                         'CDM',
                         'RDM',
                         'RWB',
                         'LB',
                         'LCB',
                         'CB',
                         'RCB',
                         'RB',
                         'GK'
                         ,"W/F",
                          "SM",
                          "IR",
                          "Value",
                          "Name",
                          "Nationality",
                          "foot",
                          "Hits"])
    fifa['Wage'] = (pd.to_numeric(fifa['Wage'].str.strip('K').str.strip('€')))* 1000
    fifa['Weight']=pd.to_numeric(fifa['Weight'].str.strip('lbs'))
    fifa['Height']=((pd.to_numeric(fifa['Height'].str[0]))*12)+ (pd.to_numeric(fifa['Height'].str.split("\'").str[1].str.strip('"')))*2.54
    fifa = fifa.dropna()
    fifa["BP"]=(fifa["BP"]
            .replace("GK",0)
            .replace("LB",1)
            .replace("CB",1)
            .replace("RB",1)
            .replace("ST",3)
            .replace("RW",3)
            .replace("LWB",3)
            .replace("LW",3)
            .replace("RWB",3)
            .replace("CDM",2)
            .replace("CAM",2)
            .replace("LM",2)
            .replace("RM",2)
            .replace("CM",2)
            .replace("CF",3))
    fifa = fifa.drop(columns=['Height','Weight','Acceleration','Sprint Speed','Agility','Balance','Jumping',
                 'Interceptions','Defending','Marking','Standing Tackle', 'Sliding Tackle', 'Goalkeeping',
                 'GK Diving','GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes', 'PAC', 'DRI','DEF','PHY']) 
    return fifa 

In [18]:
#fifa = pd.read_csv(r'/Users/larag/Documents/Ironhack/Fifa_project/fifa21_train.csv')
# fifa = pd.DataFrame(pd.read_csv('fifa21_train.csv'))
fifa = pd.DataFrame(pd.read_csv('fifa21_validate.csv'))
print(fifa.shape)    # before validate: (11701, 101)

(1999, 101)


In [19]:
fifa = fifa.drop(columns=["ID","Club"
                          ,"Position"
                          ,"Team & Contract"
                          ,"Joined"
                          ,"Loan Date End"
                          ,"Release Clause"
                          ,"Contract"
                          ,"Growth"
                          ,"A/W"
                          ,"D/W"
                          ,'LS'
                          ,'ST',
                         'RS',
                         'LW',
                         'LF',
                         'CF',
                         'RF',
                         'RW',
                         'LAM',
                         'CAM',
                         'RAM',
                         'LM',
                         'LCM',
                         'CM',
                         'RCM',
                         'RM',
                         'LWB',
                         'LDM',
                         'CDM',
                         'RDM',
                         'RWB',
                         'LB',
                         'LCB',
                         'CB',
                         'RCB',
                         'RB',
                         'GK'
                         ,"W/F",
                          "SM",
                          "IR",
                          "Value",
                          "Name",
                          "Nationality",
                          "foot",
                          "Hits"])

In [20]:
fifa['Wage'] = (pd.to_numeric(fifa['Wage'].str.strip('K').str.strip('€')))* 1000

In [21]:
fifa['Weight']=pd.to_numeric(fifa['Weight'].str.strip('lbs'))

In [22]:
fifa['Height']=((pd.to_numeric(fifa['Height'].str[0]))*12)+ (pd.to_numeric(fifa['Height'].str.split("\'").str[1].str.strip('"')))*2.54

In [23]:
fifa = fifa.dropna()

In [24]:
# GK = 0, Defense = 1, Midfield = 2, Offense = 3

fifa["BP"]=(fifa["BP"]
            .replace("GK",0)
            .replace("LB",1)
            .replace("CB",1)
            .replace("RB",1)
            .replace("ST",3)
            .replace("RW",3)
            .replace("LWB",3)
            .replace("LW",3)
            .replace("RWB",3)
            .replace("CDM",2)
            .replace("CAM",2)
            .replace("LM",2)
            .replace("RM",2)
            .replace("CM",2)
            .replace("CF",3))

In [25]:
fifa.dtypes

Age                   int64
BP                    int64
Height              float64
Weight                int64
Wage                  int64
Attacking             int64
Crossing              int64
Finishing             int64
Heading Accuracy      int64
Short Passing         int64
Volleys             float64
Skill                 int64
Dribbling             int64
Curve               float64
FK Accuracy           int64
Long Passing          int64
Ball Control          int64
Movement              int64
Acceleration          int64
Sprint Speed          int64
Agility             float64
Reactions             int64
Balance             float64
Power                 int64
Shot Power            int64
Jumping             float64
Stamina               int64
Strength              int64
Long Shots            int64
Mentality             int64
Aggression            int64
Interceptions       float64
Positioning         float64
Vision              float64
Penalties             int64
Composure           

In [26]:
fifa.corr()

,Age,BP,Height,Weight,Wage,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,...,GK Reflexes,Total Stats,Base Stats,PAC,SHO,PAS,DRI,DEF,PHY,OVA
Age,1.000000,-0.093309,-0.049150,0.245415,-0.250552,0.161731,0.124160,0.082438,0.154803,0.152380,...,0.132964,0.265483,0.406148,-0.175650,0.293637,0.384012,0.224738,0.225096,0.440184,0.511036
BP,-0.093309,1.000000,0.085510,-0.327754,-0.021139,0.715425,0.507804,0.807375,0.427856,0.515040,...,-0.622882,0.502421,0.033797,0.202646,0.313094,0.013020,0.203867,-0.329542,-0.188292,-0.016096
Height,-0.049150,0.085510,1.000000,-0.137608,-0.005397,0.102739,0.166938,0.108974,-0.042691,0.096120,...,-0.080130,0.106511,0.070427,0.145626,0.054551,0.088932,0.104460,-0.029081,-0.097900,-0.006823
Weight,0.245415,-0.327754,-0.137608,1.000000,-0.041336,-0.301580,-0.416321,-0.326636,0.045686,-0.293241,...,0.353489,-0.271523,0.011090,-0.339818,-0.015385,-0.088039,-0.201508,0.128639,0.522357,0.153761
Wage,-0.250552,-0.021139,-0.005397,-0.041336,1.000000,-0.179943,-0.153441,-0.122357,-0.161163,-0.193318,...,0.028547,-0.254430,-0.296389,-0.048704,-0.148409,-0.223261,-0.209494,-0.165037,-0.229778,-0.344150
Attacking,0.161731,0.715425,0.102739,-0.301580,-0.179943,1.000000,0.853797,0.881579,0.706756,0.887668,...,-0.772465,0.906113,0.391315,0.165308,0.313100,0.338772,0.333692,0.127482,0.074592,0.388059
Crossing,0.124160,0.507804,0.166938,-0.416321,-0.153441,0.853797,1.000000,0.673482,0.452006,0.803063,...,-0.664722,0.854196,0.444607,0.294450,0.188991,0.465588,0.402335,0.216285,-0.025786,0.347164
Finishing,0.082438,0.807375,0.108974,-0.326636,-0.122357,0.881579,0.673482,1.000000,0.445152,0.666363,...,-0.585434,0.716262,0.335145,0.269165,0.580332,0.318889,0.443499,-0.242397,-0.110508,0.263854
Heading Accuracy,0.154803,0.427856,-0.042691,0.045686,-0.161163,0.706756,0.452006,0.445152,1.000000,0.643519,...,-0.731046,0.619031,0.075322,-0.179220,-0.168480,-0.133289,-0.195308,0.408459,0.386677,0.269760
Short Passing,0.152380,0.515040,0.096120,-0.293241,-0.193318,0.887668,0.803063,0.666363,0.643519,1.000000,...,-0.734279,0.907076,0.437951,0.083052,0.123721,0.426411,0.303432,0.377462,0.139720,0.456423


In [27]:
fifa = fifa.drop(columns=['Height','Weight','Acceleration','Sprint Speed','Agility','Balance','Jumping',
                 'Interceptions','Defending','Marking','Standing Tackle', 'Sliding Tackle', 'Goalkeeping',
                 'GK Diving','GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes', 'PAC', 'DRI','DEF','PHY'])               

In [28]:
y = fifa["OVA"]
X = fifa.drop(columns=["OVA"])
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [29]:
# lm = linear_model.LinearRegression() 
# lm.fit(X_train,y_train)

In [30]:
# from sklearn.preprocessing import MinMaxScaler

# transformer = MinMaxScaler().fit(X)
# x_normalized = transformer.transform(X)
# print(x_normalized.shape)
# df1 = pd.DataFrame(x_normalized, columns=X.columns)   # (11422, 32)

In [31]:
predictions = lm.predict(X)

r2 = r2_score(y, predictions)
mse = mean_squared_error(y, predictions)
rmse = math.sqrt(mse)
mae = mean_absolute_error(y, predictions)

print('r2_train:', r2)
print('mse_train:', mse)
print('rmse_train:', rmse)
print('mae_train:', mae)


r2_train: 0.8979684966078672
mse_train: 4.615917535884187
rmse_train: 2.1484686490345135
mae_train: 1.7102594307790737


In [32]:
predictions = lm.predict(X_test)

r2 = r2_score(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = math.sqrt(mse)
mae = mean_absolute_error(y_test, predictions)

print('r2_test:', r2)
print('mse_test:', mse)
print('rmse_test:', rmse)
print('mae_test:', mae)

r2_test: 0.8843279716841483
mse_test: 4.873134028470924
rmse_test: 2.2075176167974115
mae_test: 1.768202267989874


In [33]:
def fifa_processing(fifa):
    fifa = fifa.drop(columns=["ID","Club"
                          ,"Position"
                          ,"Team & Contract"
                          ,"Joined"
                          ,"Loan Date End"
                          ,"Release Clause"
                          ,"Contract"
                          ,"Growth"
                          ,"A/W"
                          ,"D/W"
                          ,'LS'
                          ,'ST',
                         'RS',
                         'LW',
                         'LF',
                         'CF',
                         'RF',
                         'RW',
                         'LAM',
                         'CAM',
                         'RAM',
                         'LM',
                         'LCM',
                         'CM',
                         'RCM',
                         'RM',
                         'LWB',
                         'LDM',
                         'CDM',
                         'RDM',
                         'RWB',
                         'LB',
                         'LCB',
                         'CB',
                         'RCB',
                         'RB',
                         'GK'
                         ,"W/F",
                          "SM",
                          "IR",
                          "Value",
                          "Name",
                          "Nationality",
                          "foot",
                          "Hits"])
    fifa['Wage'] = (pd.to_numeric(fifa['Wage'].str.strip('K').str.strip('€')))* 1000
    fifa['Weight']=pd.to_numeric(fifa['Weight'].str.strip('lbs'))
    fifa['Height']=((pd.to_numeric(fifa['Height'].str[0]))*12)+ (pd.to_numeric(fifa['Height'].str.split("\'").str[1].str.strip('"')))*2.54
    fifa = fifa.dropna()
    fifa["BP"]=(fifa["BP"]
            .replace("GK",0)
            .replace("LB",1)
            .replace("CB",1)
            .replace("RB",1)
            .replace("ST",3)
            .replace("RW",3)
            .replace("LWB",3)
            .replace("LW",3)
            .replace("RWB",3)
            .replace("CDM",2)
            .replace("CAM",2)
            .replace("LM",2)
            .replace("RM",2)
            .replace("CM",2)
            .replace("CF",3))
    fifa = fifa.drop(columns=['Height','Weight','Acceleration','Sprint Speed','Agility','Balance','Jumping',
                 'Interceptions','Defending','Marking','Standing Tackle', 'Sliding Tackle', 'Goalkeeping',
                 'GK Diving','GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes', 'PAC', 'DRI','DEF','PHY']) 
    return fifa 